In [3]:
from google.cloud import bigquery
from google.cloud.bigquery import job
from google.cloud.bigquery import SchemaField
import os
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:\\Users\\Owner\\Downloads\\json\\ballosodeuk-4811d21b3452.json"

import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import pandas as pd
PROJCECT = 'ballosodeuk'
bq = bigquery.Client(project=PROJCECT)

# 클라이언트 설정
client = bigquery.Client()
query_name = "0711_동적쿼리테스트"
# SQL 파일 읽기
# file = './query/쿠팡0403_0509.sql'
file = f'C://FTC_downloads/code/git/project_hct/query/{query_name}.sql'
with open(file, 'r') as file:
    query = file.read()



In [129]:
import subprocess
import json
import os

def get_gcloud_account():
    try:
        # gcloud auth list 명령어를 실행하여 JSON 형식의 출력을 얻음
        result = subprocess.run(
            ['gcloud', 'auth', 'list', '--format=json'],
            capture_output=True,
            text=True,
            check=True,
            shell=True  # Windows에서는 shell=True가 필요할 수 있습니다.
        )
        # JSON 파싱
        accounts = json.loads(result.stdout)
        
        # 활성 계정 찾기
        active_account = None
        for account in accounts:
            if account.get('status') == 'ACTIVE':
                active_account = account.get('account')
                break
        
        return active_account

    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")
        return None
    except FileNotFoundError as e:
        print("gcloud 명령어를 찾을 수 없습니다. gcloud SDK가 설치되었는지 확인하고 PATH에 추가하세요.")
        return None

# 현재 gcloud 로그인 계정 확인
current_account = get_gcloud_account()
if current_account:
    print(f"현재 gcloud 로그인 계정: {current_account}")
else:
    print("gcloud 로그인 계정을 확인할 수 없습니다.")


현재 gcloud 로그인 계정: balsohecto@gmail.com


In [7]:

# 쿼리 실행
job_config = bigquery.QueryJobConfig()
query_job = client.query(query, job_config=job_config)

df_2 = query_job.to_dataframe()

Forbidden: 403 POST https://bigquery.googleapis.com/bigquery/v2/projects/ballosodeuk/jobs?prettyPrint=false: Access Denied: Project ballosodeuk: User does not have bigquery.jobs.create permission in project ballosodeuk.

Location: None
Job ID: 6070f00a-bb42-45dd-bcce-2b5123e9c573


In [ ]:
df_2

,Event_Date,Airbridge_Device_ID,Airbridge_Device_ID_Type,i,Event_Category,Event_Value_Total,Event_Count_Total,e


### 스키마 확인 및 업데이트 

In [5]:
# 데이터셋 및 테이블 ID 설정
dataset_id = 'airbridge_mart'
table_id = 'app_df_2'
table_ref = f'ballosodeuk.{dataset_id}.{table_id}'


In [33]:
def infer_schema_from_dataframe(df):
    """데이터프레임에서 스키마 추론."""
    def infer_field_type(value):
        if isinstance(value, dict):
            subfields = {k: infer_field_type(v) for k, v in value.items()}
            return subfields
        elif isinstance(value, list) and len(value) > 0 and isinstance(value[0], dict):
            subfields = {k: infer_field_type(v) for k, v in value[0].items()}
            return {"type": "RECORD", "mode": "REPEATED", "fields": subfields}
        elif pd.api.types.is_integer_dtype(type(value)):
            return "INTEGER"
        elif pd.api.types.is_float_dtype(type(value)):
            return "FLOAT"
        elif pd.api.types.is_bool_dtype(type(value)):
            return "BOOLEAN"
        else:
            return "STRING"
    
    schema = {col: infer_field_type(df[col].iloc[0]) for col in df.columns}
    return schema

In [16]:
def get_current_schema(table_ref):
    """기존 테이블의 스키마를 가져옵니다."""
    table = client.get_table(table_ref)
    return {field.name: field for field in table.schema}

In [17]:
def update_table_schema(table_ref, new_columns):
    """테이블 스키마에 새 열을 추가합니다."""
    current_schema = get_current_schema(table_ref)
    new_fields = []

    for name, dtype in new_columns.items():
        if name not in current_schema:
            if isinstance(dtype, dict):
                # STRUCT 필드 처리
                subfields = [SchemaField(subname, subtype) for subname, subtype in dtype.items()]
                new_fields.append(SchemaField(name, "RECORD", fields=subfields))
            else:
                new_fields.append(SchemaField(name, dtype))

    if new_fields:
        table = client.get_table(table_ref)
        updated_schema = table.schema + new_fields
        table.schema = updated_schema
        client.update_table(table, ["schema"])
        print(f"테이블 {table_ref}의 스키마가 업데이트되었습니다: {new_columns}")

In [9]:
def load_data_to_bigquery(df, table_ref):
    """데이터프레임을 빅쿼리 테이블로 업로드합니다."""
    job = client.load_table_from_dataframe(df, table_ref)
    job.result()
    print(f"테이블 {table_ref}에 데이터가 성공적으로 업로드되었습니다.")

In [34]:
new_columns = infer_schema_from_dataframe(df)
current_schema = get_current_schema(table_ref)

In [35]:
df['e'].iloc[0][0]['d']

array([{'Timestamp': '2024-07-09T01:47:59+09:00', 'Label': '챌린지', 'Action': '챌린지둘러보기', 'Value': 150.0, 'contributionMargin': None, 'datetime': None, 'listID': None, 'originalContributionMargin': None, 'place': None, 'rate': None, 'scheduleID': None, 'score': None, 'type': None, 'sharedChannel': None, 'achievementID': None, 'tester': 'true', 'campaignName': None, 'campaignType': None, 'commission': None, 'gettableCash': None, '_field_21': None, 'incentive_product_name': None, 'products_struct_alias': array([], dtype=object)}],
      dtype=object)

In [37]:
df

,Event_Date,Airbridge_Device_ID,Airbridge_Device_ID_Type,i,Event_Category,Event_Value_Total,Event_Count_Total,e
0,2024-07-09,5e460020-fc23-44f6-8100-e8d4a00390c1,GAID,"{'Device_Model': 'SM-A156L', 'Device_Type': 'm...",view_get__reward_done (App),150.0,1,"[{'Label': '챌린지', 'Action': '챌린지둘러보기', 'Event_..."
1,2024-07-09,12405fb3-fcc6-488b-a62f-20f74e703902,GAID,"{'Device_Model': 'SM-S908N', 'Device_Type': 'm...",view_get__reward_done (App),-22.0,2,"[{'Label': '오퍼월', 'Action': '애드팝콘', 'Event_Val..."
2,2024-07-09,d3200538-8e0a-4d2b-88f2-3025586236fa,GAID,"{'Device_Model': 'null', 'Device_Type': 'mobil...",view_get__reward_done (App),1.0,1,"[{'Label': '오퍼월', 'Action': '애드팝콘', 'Event_Val..."
3,2024-07-09,e450347f-c22d-405c-be68-ba110a639532,GAID,"{'Device_Model': 'SM-N976N', 'Device_Type': 'm...",view_get__reward_done (App),18.0,2,"[{'Label': '오퍼월', 'Action': '애드팝콘', 'Event_Val..."
4,2024-07-09,bf24a645-a71b-409b-a424-5842252a3041,GAID,"{'Device_Model': 'null', 'Device_Type': 'mobil...",view_get__reward_done (App),1.0,1,"[{'Label': '오퍼월', 'Action': '애드팝콘', 'Event_Val..."
...,...,...,...,...,...,...,...,...
8331,2024-07-09,8721a665-8c01-4996-a59d-358688941953,GAID,"{'Device_Model': 'SM-A516N', 'Device_Type': 'm...",view_get__reward_done (App),4.0,2,"[{'Label': '박터뜨리기', 'Action': '박터뜨리기', 'Event_..."
8332,2024-07-09,f02ff5cc-6020-40cc-a20e-f2f5b99c3c37,GAID,"{'Device_Model': 'SM-A716S', 'Device_Type': 'm...",view_get__reward_done (App),20.0,1,"[{'Label': '보너스봉투_메인', 'Action': '보너스봉투', 'Eve..."
8333,2024-07-09,02295a8b-4bce-4554-8eea-f7da23c51755,GAID,"{'Device_Model': 'SM-A326K', 'Device_Type': 'm...",view_get__reward_done (App),18.0,1,"[{'Label': '보너스봉투_메인', 'Action': '보너스봉투', 'Eve..."
8334,2024-07-09,fa139c31-ed99-45c1-ad38-81b957b14237,GAID,"{'Device_Model': 'null', 'Device_Type': 'mobil...",view_get__reward_done (App),1.0,1,"[{'Label': '오퍼월', 'Action': '애드팝콘', 'Event_Val..."


In [22]:
current_schema

{'Event_Date': SchemaField('Event_Date', 'DATE', 'NULLABLE', None, (), None),
 'Airbridge_Device_ID': SchemaField('Airbridge_Device_ID', 'STRING', 'NULLABLE', None, (), None),
 'Airbridge_Device_ID_Type': SchemaField('Airbridge_Device_ID_Type', 'STRING', 'NULLABLE', None, (), None),
 'i': SchemaField('i', 'RECORD', 'NULLABLE', None, (SchemaField('Device_Model', 'STRING', 'NULLABLE', None, (), None), SchemaField('Device_Type', 'STRING', 'NULLABLE', None, (), None), SchemaField('Platform', 'STRING', 'NULLABLE', None, (), None), SchemaField('Client_IP_Country_Code', 'STRING', 'NULLABLE', None, (), None), SchemaField('Client_IP_Subdivision', 'STRING', 'NULLABLE', None, (), None), SchemaField('Client_IP_City', 'STRING', 'NULLABLE', None, (), None), SchemaField('Campaign_ID', 'STRING', 'NULLABLE', None, (), None), SchemaField('Ad_Group_ID', 'STRING', 'NULLABLE', None, (), None), SchemaField('Ad_Creative_ID', 'STRING', 'NULLABLE', None, (), None), SchemaField('Term_ID', 'STRING', 'NULLABLE', 

In [23]:
# 새 데이터의 스키마 추론 및 기존 스키마 비교 및 업데이트
new_columns = infer_schema_from_dataframe(df)
current_schema = get_current_schema(table_ref)
missing_columns = {col: dtype for col, dtype in new_columns.items() if col not in current_schema}

if missing_columns:
    update_table_schema(table_ref, missing_columns)

# 데이터 업로드
load_data_to_bigquery(df, table_ref)

{'Airbridge_Device_ID',
 'Airbridge_Device_ID_Type',
 'Event_Category',
 'Event_Count_Total',
 'Event_Date',
 'Event_Value_Total',
 'e',
 'i'}

In [ ]:
if missing_columns:
    update_table_schema(table_ref, missing_columns)

In [45]:

# # 예제 데이터프레임 (매일 추가되는 데이터)
# new_data = {
#     "Event_Date": ["2024-07-10"],
#     "Airbridge_Device_ID": ["abcd1234"],
#     "Airbridge_Device_ID_Type": ["type1"],
#     "i": [{"Device_Model": "model1", "Device_Type": "type1"}],
#     "Event_Category": ["category1"],
#     "Event_Value_Total": [123.45],
#     "Event_Count_Total": [1],
#     "e": [[{
#         "Label": "label1", 
#         "Action": "action1", 
#         "Event_Value_Sum": 12.34, 
#         "Event_Count": 1,
#         "u": [{"User_ID": "user1"}],
#         "d": [{"Timestamp": "2024-07-09T01:47:59+09:00", "Label": "챌린지", "Action": "챌린지둘러보기", "Value": 150.0, "contributionMargin": None, "datetime": None, "listID": None, "originalContributionMargin": None, "place": None, "rate": None, "scheduleID": None, "score": None, "type": None, "sharedChannel": None, "achievementID": None, "tester": 'true', 'campaignName': None, 'campaignType': None, 'commission': None, 'gettableCash': None, '_field_21': None, 'incentive_product_name': None, 'products_struct_alias': []}]
#     }]]
# }
# df = pd.DataFrame(new_data)

def infer_schema_from_dataframe(df):
    """데이터프레임에서 스키마 추론."""
    def infer_field_type(value):
        if isinstance(value, dict):
            subfields = {k: infer_field_type(v) for k, v in value.items()}
            return subfields
        elif isinstance(value, list) and len(value) > 0 and isinstance(value[0], dict):
            subfields = {k: infer_field_type(v) for k, v in value[0].items()}
            return {"type": "RECORD", "mode": "REPEATED", "fields": subfields}
        elif pd.api.types.is_integer_dtype(type(value)):
            return "INTEGER"
        elif pd.api.types.is_float_dtype(type(value)):
            return "FLOAT"
        elif pd.api.types.is_bool_dtype(type(value)):
            return "BOOLEAN"
        else:
            return "STRING"
    
    schema = {col: infer_field_type(df[col].iloc[0]) for col in df.columns}
    return schema

new_columns = infer_schema_from_dataframe(df)
current_schema = get_current_schema(table_ref)

new_columns

{'Event_Date': 'STRING',
 'Airbridge_Device_ID': 'STRING',
 'Airbridge_Device_ID_Type': 'STRING',
 'i': {'Device_Model': 'STRING',
  'Device_Type': 'STRING',
  'Platform': 'STRING',
  'Client_IP_Country_Code': 'STRING',
  'Client_IP_Subdivision': 'STRING',
  'Client_IP_City': 'STRING',
  'Campaign_ID': 'STRING',
  'Ad_Group_ID': 'STRING',
  'Ad_Creative_ID': 'STRING',
  'Term_ID': 'STRING',
  'Is_Re_engagement': 'STRING',
  'Is_First_Event_per_User_ID': 'BOOLEAN',
  'Is_First_Event_per_Device_ID': 'BOOLEAN',
  'Is_First_Target_Event_per_Device': 'BOOLEAN',
  'Target_Event_Timestamp': 'STRING',
  'Target_Event_Category': 'STRING'},
 'Event_Category': 'STRING',
 'Event_Value_Total': 'FLOAT',
 'Event_Count_Total': 'INTEGER',
 'e': 'STRING'}

In [52]:
df.head()['e'][0]

array([{'Label': '챌린지', 'Action': '챌린지둘러보기', 'Event_Value_Sum': 150.0, 'Event_Count': 1, 'u': array([{'User_ID': '7b1acc51-3759-4ace-8f7b-fb6312e9d889'}], dtype=object), 'd': array([{'Timestamp': '2024-07-09T01:47:59+09:00', 'Label': '챌린지', 'Action': '챌린지둘러보기', 'Value': 150.0, 'tester': 'true', 'campaignName': None, 'campaignType': None, 'commission': None, 'gettableCash': None, '_field_10': None, 'incentive_product_name': None, 'contributionMargin': None, 'datetime': None, 'listID': None, 'originalContributionMargin': None, 'place': None, 'rate': None, 'scheduleID': None, 'score': None, 'type': None, 'sharedChannel': None, 'achievementID': None, 'products_struct_alias': array([], dtype=object)}],
             dtype=object)}                                                                                                                                                                                                                                                                          

In [104]:
df['e'][0][0].keys()
df['e'][0][0]['d'][0].keys()
df['e'][1002][0]['d'][0]['products_struct_alias']

array([], dtype=object)

스키마 추론

In [79]:
def infer_schema_from_dataframe(df):
    """데이터프레임에서 스키마 추론."""
    def infer_field_type(value):
        if isinstance(value, dict):
            subfields = {k: infer_field_type(v) for k, v in value.items()}
            return {"type": "RECORD", "fields": subfields}
        elif isinstance(value, (np.ndarray, list)) and len(value) > 0 and isinstance(value[0], dict):
            subfields = {k: infer_field_type(v) for k, v in value[0].items()}
            return {"type": "RECORD", "mode": "REPEATED", "fields": subfields}
        elif isinstance(value, (np.ndarray, list)):
            return {"type": "STRING", "mode": "REPEATED"}
        elif pd.api.types.is_integer_dtype(type(value)):
            return "INTEGER"
        elif pd.api.types.is_float_dtype(type(value)):
            return "FLOAT"
        elif pd.api.types.is_bool_dtype(type(value)):
            return "BOOLEAN"
        elif pd.api.types.is_datetime64_any_dtype(type(value)):
            return "TIMESTAMP"
        else:
            return "STRING"
    
    schema = {col: infer_field_type(df[col].iloc[0]) for col in df.columns}
    return schema

In [80]:
df.columns

Index(['Event_Date', 'Airbridge_Device_ID', 'Airbridge_Device_ID_Type', 'i',
       'Event_Category', 'Event_Value_Total', 'Event_Count_Total', 'e'],
      dtype='object')

In [81]:
infer_schema_from_dataframe(df)

{'Event_Date': 'STRING',
 'Airbridge_Device_ID': 'STRING',
 'Airbridge_Device_ID_Type': 'STRING',
 'i': {'type': 'RECORD',
  'fields': {'Device_Model': 'STRING',
   'Device_Type': 'STRING',
   'Platform': 'STRING',
   'Client_IP_Country_Code': 'STRING',
   'Client_IP_Subdivision': 'STRING',
   'Client_IP_City': 'STRING',
   'Campaign_ID': 'STRING',
   'Ad_Group_ID': 'STRING',
   'Ad_Creative_ID': 'STRING',
   'Term_ID': 'STRING',
   'Is_Re_engagement': 'STRING',
   'Is_First_Event_per_User_ID': 'BOOLEAN',
   'Is_First_Event_per_Device_ID': 'BOOLEAN',
   'Is_First_Target_Event_per_Device': 'BOOLEAN',
   'Target_Event_Timestamp': 'STRING',
   'Target_Event_Category': 'STRING'}},
 'Event_Category': 'STRING',
 'Event_Value_Total': 'FLOAT',
 'Event_Count_Total': 'INTEGER',
 'e': {'type': 'RECORD',
  'mode': 'REPEATED',
  'fields': {'Label': 'STRING',
   'Action': 'STRING',
   'Event_Value_Sum': 'FLOAT',
   'Event_Count': 'INTEGER',
   'u': {'type': 'RECORD',
    'mode': 'REPEATED',
    'fie

In [107]:
current_schema

{'Event_Date': SchemaField('Event_Date', 'DATE', 'NULLABLE', None, (), None),
 'Airbridge_Device_ID': SchemaField('Airbridge_Device_ID', 'STRING', 'NULLABLE', None, (), None),
 'Airbridge_Device_ID_Type': SchemaField('Airbridge_Device_ID_Type', 'STRING', 'NULLABLE', None, (), None),
 'i': SchemaField('i', 'RECORD', 'NULLABLE', None, (SchemaField('Device_Model', 'STRING', 'NULLABLE', None, (), None), SchemaField('Device_Type', 'STRING', 'NULLABLE', None, (), None), SchemaField('Platform', 'STRING', 'NULLABLE', None, (), None), SchemaField('Client_IP_Country_Code', 'STRING', 'NULLABLE', None, (), None), SchemaField('Client_IP_Subdivision', 'STRING', 'NULLABLE', None, (), None), SchemaField('Client_IP_City', 'STRING', 'NULLABLE', None, (), None), SchemaField('Campaign_ID', 'STRING', 'NULLABLE', None, (), None), SchemaField('Ad_Group_ID', 'STRING', 'NULLABLE', None, (), None), SchemaField('Ad_Creative_ID', 'STRING', 'NULLABLE', None, (), None), SchemaField('Term_ID', 'STRING', 'NULLABLE', 

In [ ]:
def infer_field_type(value):
    if isinstance(value, dict):
        subfields = {k: infer_field_type(v) for k, v in value.items()}
        return {"type": "RECORD", "fields": subfields}
    elif isinstance(value, (np.ndarray, list)) and len(value) > 0 and isinstance(value[0], dict):
        subfields = {k: infer_field_type(v) for k, v in value[0].items()}
        return {"type": "RECORD", "mode": "REPEATED", "fields": subfields}
    elif isinstance(value, (np.ndarray, list)):
        return {"type": "STRING", "mode": "REPEATED"}
    elif pd.api.types.is_integer_dtype(type(value)):
        return "INTEGER"
    elif pd.api.types.is_float_dtype(type(value)):
        return "FLOAT"
    elif pd.api.types.is_bool_dtype(type(value)):
        return "BOOLEAN"
    elif pd.api.types.is_datetime64_any_dtype(type(value)):
        return "TIMESTAMP"
    else:
        return "STRING"

def merge_schemas(schema1, schema2):
    if schema1["type"] != schema2["type"]:
        return "STRING"
    if schema1["type"] == "RECORD":
        merged_fields = schema1["fields"]
        for k, v in schema2["fields"].items():
            if k in merged_fields:
                merged_fields[k] = merge_schemas(merged_fields[k], v)
            else:
                merged_fields[k] = v
        return {"type": "RECORD", "fields": merged_fields}
    return schema1

def infer_schema_from_dataframe(df):
    """데이터프레임에서 스키마 추론."""
    schema = {}

    for col in df.columns:
        col_schema = None
        for value in df[col]:
            inferred_type = infer_field_type(value)
            if col_schema is None:
                col_schema = inferred_type
            else:
                col_schema = merge_schemas(col_schema, inferred_type)
        schema[col] = col_schema

    return schema